# Recopilación de datos meteorológicos de una API

Este cuaderno contiene el código que se utilizó para recopilar los datos de este capítulo. Ten en cuenta que si sobrescribes los datos que venían con este capítulo guardando los datos que recopiles aquí, tus resultados en los cuadernos restantes pueden no coincidir con el libro debido a cambios en los datos de la API del NCEI.

## Acerca de los datos
En este cuaderno recopilaremos datos meteorológicos diarios del [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). Utilizaremos el conjunto de datos Global Historical Climatology Network - Daily (GHCND); véase la documentación [here](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Uso de la API del NCEI
Solicite su token [here](https://www.ncdc.noaa.gov/cdo-web/token) y pégalo a continuación.

In [10]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv('NOAA_TOKEN')

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': TOKEN
        },
        params=payload
    )

## Recopilar todos los puntos de datos de 2018 en NYC (varias estaciones)
Podemos hacer un bucle para consultar todos los puntos de datos un día a la vez, proporcionando actualizaciones usando `IPython.display`. Aquí creamos una lista con todos los resultados:

In [14]:
import datetime

from IPython import display # para actualizar la celda dinámicamente

current = datetime.date(2018, 1, 1)
end = datetime.date(2019, 1, 1)

results = []

while current < end:
    # actualizar la célula con la información de estado
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    
    response = make_request(
        'data', 
        {
            'datasetid': 'GHCND', # Conjunto de datos de la Red Mundial de Climatología Histórica Diaria (GHCND)
            'locationid': 'CITY:US360019', # NYC
            'startdate': current,
            'enddate': current,
            'units': 'metric',
            'limit': 1000 # máximo permitido
        }
    )

    if response.ok:
        # extendemos la lista en lugar de anexar para evitar obtener una lista anidada
        results.extend(response.json()['results'])

    # actualizar la fecha actual para evitar un bucle infinito
    current += datetime.timedelta(days=1)

'Gathering data for 2018-12-31'

Ahora, podemos crear un marco de datos con todos estos datos. Observa que hay varias estaciones con valores para cada `datatype` en un día determinado. No sabemos cuáles son las estaciones, pero podemos buscarlas y añadirlas a los datos:

In [15]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,0800",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,1050",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,1050",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,0920",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,0920",0.0


Guarda estos datos en un archivo:

In [16]:
df.to_csv('data/nyc_weather_2018.csv', index=False)

y escribirlo en la base de datos:

In [17]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    df.to_sql(
        'weather', connection, index=False, if_exists='replace'
    )

Para aprender a fusionar marcos de datos, también obtendremos los datos que relacionan los ID de estación con información sobre la estación:

In [18]:
response = make_request(
    'stations', 
    {
        'datasetid': 'GHCND', # Conjunto de datos de la Red Mundial de Climatología Histórica Diaria (GHCND)
        'locationid': 'CITY:US360019', # NYC
        'limit': 1000 #máximo permitido
    }
)

stations = pd.DataFrame(response.json()['results'])[['id', 'name', 'latitude', 'longitude', 'elevation']]
stations.to_csv('data/weather_stations.csv', index=False)

with sqlite3.connect('data/weather.db') as connection:
    stations.to_sql(
        'stations', connection, index=False, if_exists='replace'
    )

<hr>
<div>
    <a href="../ch_03/5-handling_data_issues.ipynb">
        <button>&#8592; Capitulo 3</button>
    </a>
    <a href="./1-querying_and_merging.ipynb">
        <button style="float: right;">Proximo Notebook &#8594;</button>
    </a>
</div>
<hr>